In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np
from google.colab import files
import io

In [2]:
model = tf.keras.models.load_model('nsfw_classification_fine_tuned.keras')

In [3]:
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)           │ (None, 180, 180, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential_2 (Sequential)            │ (None, 180, 180, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ true_divide (TrueDivide)             │ (None, 180, 180, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ subtract (Subtract)                  │ (None, 180, 180, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ mobilenetv2_1.00_224 (Functional)    │ (None, 6, 6, 1280)          │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 1280)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 5)                   │           6,405 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,132,236 (15.76 MB)

 Trainable params: 1,867,845 (7.13 MB)

 Non-trainable params: 396,544 (1.51 MB)

 Optimizer params: 1,867,847 (7.13 MB)

In [4]:
def fix_model(model):
    input_layer = tf.keras.layers.Input(shape=(180, 180, 3))

    x = model.layers[1](input_layer)

    for layer in model.layers[2:]:
        x = layer(x)

    fixed_model = tf.keras.models.Model(inputs=input_layer, outputs=x)
    return fixed_model

In [5]:
model = fix_model(model)

In [6]:
img = image.load_img('test_image_sexy.jpg', target_size=(180, 180))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = img_array / 255.0

In [7]:
predictions = model.predict(img_array)

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


In [8]:
class_names = ['drawings', 'hentai', 'neutral', 'porn', 'sexy']
predicted_class = class_names[np.argmax(predictions)]
confidence = np.max(predictions)

print(f"Predicted class: {predicted_class} with confidence {confidence:.2f}")

Predicted class: sexy with confidence 5.41


In [9]:
predictions = model.predict(img_array)
softmax_predictions = tf.nn.softmax(predictions).numpy()
predicted_class = np.argmax(softmax_predictions)
predicted_class_prob = np.max(softmax_predictions)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step


In [10]:
print(f"Predicted Class: {predicted_class}")
print(f"Prediction Probability: {predicted_class_prob:.4f}")

Predicted Class: 4
Prediction Probability: 0.9963
